# California Grants Analysis: 2nd Notebook

### Joshua Susanto 8/18/2022 

**This notebook will further explore the California grants dataset. Part 1 includes:**
 - cleaning the dataset
 - implementation of numeric variables
 - initial EDA of numeric variables
 - analysis on grant agencies and ongoing status of grants
 - function to table unique categories for all grants
 
**Goals of this notebook include:**
- exploring which agency gives the most money per grant
- implement binary columns for each unique category per entry
- keyword analysis of the grants' descriptions 
- exploring the different general topic per subdivision of keywords

In [1]:
# import necessary packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# data we will be using
grants = pd.read_csv("ca_grants_clean.csv")
grants_ongoing = pd.read_csv("grants_ongoing.csv")

**Exploring which agency gives the most money per grant**

In [3]:
print(grants_ongoing.MaxAmounts.groupby(grants_ongoing.AgencyDept).mean().sort_values(ascending = False).head(10))

AgencyDept
State Treasurer's Office                            1.193111e+09
Department of Transportation                        1.000000e+07
State Water Resources Control Board                 2.941667e+06
Department of Pesticide Regulation                  2.325000e+06
Strategic Growth Council                            1.750000e+06
Department of Toxic Substances Control              9.833333e+05
Coachella Valley Mountains Conservancy              4.000000e+05
Department of Parks and Recreation                  2.550000e+05
Governor's Office of Emergency Services             2.255000e+05
Department of Health Care Access and Information    5.000000e+04
Name: MaxAmounts, dtype: float64


The state treasurer's office gives out the most money per grant on average. Looking specificaly at this department we can see that a majority of these are actually loans and most don't even have reported amounts

In [4]:
grants_ongoing[grants_ongoing.AgencyDept == 'State Treasurer\'s Office']

,PortalID,Status,LastUpdated,AgencyDept,Title,Type,LOI,Categories,Purpose,Description,...,FundingMethod,OpenDate,ApplicationDeadline,ExpAwardDate,GrantURL,MaxAwards,MinAwards,MaxAmounts,MinAmounts,IsOngoing
51,1596,active,2022-02-23 00:21:26,State Treasurer's Office,Children&#039;s Hospital Program of 2018 -Chil...,Grant,No,Health & Human Services,The purpose of the program is to improve the h...,"On November 6, 2018, California voters passed ...",...,Advances & Reimbursement(s),2022-02-22 08:00:00,Ongoing,Ongoing,https://www.treasurer.ca.gov/chffa/hospital.asp,NaN,NaN,1.350000e+08,1.0,1
93,1848,active,2021-07-09 16:35:51,State Treasurer's Office,California Pollution Control Financing Authori...,Loan,No,Energy; Environment & Water,The Pollution Control Tax-Exempt Bond Financin...,CPCFA acts as a conduit issuer in the transact...,...,Advances & Reimbursement(s),2020-07-09 07:00:00,Ongoing,Ongoing,https://www.treasurer.ca.gov/cpcfa/tax_exempt.asp,NaN,NaN,5.500000e+08,1500000.0,1
94,1842,active,2020-07-09 22:47:47,State Treasurer's Office,California Capital Access Program (CalCAP) Sei...,Loan,No,Disaster Prevention & Relief,The CalCAP/Seismic Safety Financing Program is...,The loans in this program must be used to help...,...,Reimbursement(s),2020-07-09 07:00:00,Ongoing,On a rolling basis depending on the lender,https://www.treasurer.ca.gov/cpcfa/calcap/seis...,NaN,NaN,NaN,NaN,1
95,1836,active,2021-12-20 19:31:24,State Treasurer's Office,California Capital Access Program (CalCAP) Ame...,Loan,No,"Disadvantaged Communities; Housing, Community ...",The program is designed to assist small busine...,The CalCAP for ADA Program encourages banks an...,...,Reimbursement(s),2020-07-09 07:00:00,Ongoing,On a rolling basis,https://www.treasurer.ca.gov/cpcfa/calcap/ada/...,NaN,NaN,NaN,NaN,1
96,1809,active,2021-12-20 19:28:42,State Treasurer's Office,California Capital Access Program (CalCAP) Col...,Loan,No,"Disadvantaged Communities; Housing, Community ...",The program is designed to pledge cash to cove...,Loans may be for any small business purpose wi...,...,Reimbursement(s),2020-07-09 07:00:00,Ongoing,On a rolling basis,https://www.treasurer.ca.gov/cpcfa/calcap/coll...,NaN,NaN,NaN,NaN,1
97,1806,active,2020-07-28 16:11:40,State Treasurer's Office,California Capital Access Program (CalCAP) Cal...,Loan,No,Environment & Water; Transportation,The program is designed to help borrowers fina...,Loans in this program must be used for the pur...,...,Reimbursement(s),2020-07-09 07:00:00,Ongoing,On a rolling basis,https://www.treasurer.ca.gov/cpcfa/calcap/arb/...,NaN,NaN,NaN,NaN,1
98,1803,active,2021-12-20 19:33:47,State Treasurer's Office,California Capital Access Program (CalCAP) for...,Loan,No,"Disadvantaged Communities; Housing, Community ...",The CalCAP for Small Business Program encourag...,CalCAP is a loan loss reserve program which ma...,...,Reimbursement(s),2020-07-09 07:00:00,Ongoing,On a rolling basis,https://www.treasurer.ca.gov/cpcfa/calcap/sb/i...,NaN,NaN,NaN,NaN,1
100,1584,active,2022-07-12 18:19:49,State Treasurer's Office,CHFFA Bond Financing Program,Loan,No,Health & Human Services,This program will provide a borrower with acce...,Eligibility General Requirements -Must be a ...,...,Advances & Reimbursement(s),2020-07-07 22:23:24,Ongoing,Ongoing,https://www.treasurer.ca.gov/chffa/programs/bo...,NaN,NaN,5.000000e+09,5000000.0,1
102,1662,active,2020-07-07 18:16:08,State Treasurer's Office,CEFA Bond Financing Program,Loan,No,Education,This program will provide a borrower with acce...,Eligibility General Requirements -Be regiona...,...,Advances & Reimbursement(s),2020-07-06 07:00:00,Ongoing,Ongoing,https://www.treasurer.ca.gov/cefa/bond.asp,NaN,NaN,5.000000e+09,5000000.0,1
103,1641,active,2022-07-13 20:02:33,State Treasurer's Office,Healthcare Expansion Loan Program II (HELP II),Loan,No,Health & Human Services,This program will provide low-interest rate lo...,Eligibility -Must be a health facility as def...,...,Advance(s),2020-07-06 07:00:00

In [5]:
# dataset with no missing grant values
grants_ongoing2 = grants_ongoing.dropna(subset = 'MaxAmounts')
print(grants_ongoing2.MaxAmounts.groupby(grants_ongoing.AgencyDept).mean().sort_values(ascending = False).head(10))
grants_ongoing2[grants_ongoing2.AgencyDept == 'State Treasurer\'s Office']

AgencyDept
State Treasurer's Office                            1.193111e+09
Department of Transportation                        1.000000e+07
State Water Resources Control Board                 2.941667e+06
Department of Pesticide Regulation                  2.325000e+06
Strategic Growth Council                            1.750000e+06
Department of Toxic Substances Control              9.833333e+05
Coachella Valley Mountains Conservancy              4.000000e+05
Department of Parks and Recreation                  2.550000e+05
Governor's Office of Emergency Services             2.255000e+05
Department of Health Care Access and Information    5.000000e+04
Name: MaxAmounts, dtype: float64


,PortalID,Status,LastUpdated,AgencyDept,Title,Type,LOI,Categories,Purpose,Description,...,FundingMethod,OpenDate,ApplicationDeadline,ExpAwardDate,GrantURL,MaxAwards,MinAwards,MaxAmounts,MinAmounts,IsOngoing
51,1596,active,2022-02-23 00:21:26,State Treasurer's Office,Children&#039;s Hospital Program of 2018 -Chil...,Grant,No,Health & Human Services,The purpose of the program is to improve the h...,"On November 6, 2018, California voters passed ...",...,Advances & Reimbursement(s),2022-02-22 08:00:00,Ongoing,Ongoing,https://www.treasurer.ca.gov/chffa/hospital.asp,NaN,NaN,1.350000e+08,1.0,1
93,1848,active,2021-07-09 16:35:51,State Treasurer's Office,California Pollution Control Financing Authori...,Loan,No,Energy; Environment & Water,The Pollution Control Tax-Exempt Bond Financin...,CPCFA acts as a conduit issuer in the transact...,...,Advances & Reimbursement(s),2020-07-09 07:00:00,Ongoing,Ongoing,https://www.treasurer.ca.gov/cpcfa/tax_exempt.asp,NaN,NaN,5.500000e+08,1500000.0,1
100,1584,active,2022-07-12 18:19:49,State Treasurer's Office,CHFFA Bond Financing Program,Loan,No,Health & Human Services,This program will provide a borrower with acce...,Eligibility General Requirements -Must be a ...,...,Advances & Reimbursement(s),2020-07-07 22:23:24,Ongoing,Ongoing,https://www.treasurer.ca.gov/chffa/programs/bo...,NaN,NaN,5.000000e+09,5000000.0,1
102,1662,active,2020-07-07 18:16:08,State Treasurer's Office,CEFA Bond Financing Program,Loan,No,Education,This program will provide a borrower with acce...,Eligibility General Requirements -Be regiona...,...,Advances & Reimbursement(s),2020-07-06 07:00:00,Ongoing,Ongoing,https://www.treasurer.ca.gov/cefa/bond.asp,NaN,NaN,5.000000e+09,5000000.0,1
103,1641,active,2022-07-13 20:02:33,State Treasurer's Office,Healthcare Expansion Loan Program II (HELP II),Loan,No,Health & Human Services,This program will provide low-interest rate lo...,Eligibility -Must be a health facility as def...,...,Advance(s),2020-07-06 07:00:00,Ongoing,7/6/2020,https://www.treasurer.ca.gov/chffa/programs/he...,NaN,NaN,1.500000e+06,25000.0,1
104,1632,active,2020-07-07 18:15:20,State Treasurer's Office,Tax-Exempt Equipment Financing Program,Loan,No,Health & Human Services,This program will provide a borrower with acce...,Eligibility General Requirements -Must be a ...,...,Advances & Reimbursement(s),2020-07-06 07:00:00,Ongoing,7/6/2020,https://www.treasurer.ca.gov/chffa/programs/ta...,NaN,NaN,5.000000e+07,500000.0,1
106,1494,active,2020-07-03 20:48:47,State Treasurer's Office,COVID-19 Emergency HELP Loan Program,Loan,No,Disaster Prevention & Relief; Health & Human S...,This program will provide low cost loans with ...,Eligibility General Requirements: -Annual gr...,...,Advances & Reimbursement(s),2020-07-01 07:00:00,Ongoing,Ongoing,https://www.treasurer.ca.gov/chffa/covid19-pro...,20.0,0.0,2.500000e+05,0.0,1
117,1962,active,2020-07-10 23:26:29,State Treasurer's Office,Project Acceleration Notes and Credit Enhancem...,Loan,No,Education,Designed to help provide credit enhancement to...,The Project Acceleration Notes and Credit Enha...,...,Advance(s),2017-07-01 07:00:00,Ongoing,Ongoing,https://www.treasurer.ca.gov/csfa/panacea/inde...,NaN,NaN,1.000000e+06,0.0,1
120,1902,active,2022-07-08 16:35:22,State Treasurer's Office,The Charter School Revolving Loan Fund Program,Loan,No,Education,The Charter School Revolving Loan Fund Program...,The Charter School Revolving Loan Fund (CSRFL)...,...,Advance(s),2013-07-01 07:00:00,Ongoing,Pending,https://www.treasurer.ca.gov/csfa/csrlf/index.asp,NaN,NaN,2.500000e+05,100000.0,1


We can see that removing missing values gives us only 9 entries for the state treasury. Additionally, 8 out of the 9 of these entries are loans rather than grants. 

Below we can see that there is a large difference on average between the funding/amounts of grants vs loans. With loans on average having higher funding by a factor of 10 and giving out more money by a factor of 100. This may explain why the State Treasury is at the top of our list with mainly loans.

In [6]:
print(grants_ongoing.groupby('Type').mean())
grants_ongoing.groupby('Type').size()

                PortalID  EstAvailFunds  MaxAwards  MinAwards    MaxAmounts  \
Type                                                                          
Grant        7451.255556   6.257919e+07       20.8  12.666667  1.001944e+07   
Grant; Loan  3731.100000   2.024337e+08        3.0   3.000000  5.250000e+06   
Loan         3384.600000   4.417392e+08       20.0   0.000000  1.060560e+09   

               MinAmounts  IsOngoing  
Type                                  
Grant        7.381281e+04        1.0  
Grant; Loan  1.125000e+06        1.0  
Loan         1.212500e+06        1.0  


Type
Grant          90
Grant; Loan    10
Loan           25
dtype: int64

**Creating Binary columns for Unique Categories**

In [7]:
grants.Categories[23] # need to account for the space after the semicolon -> split at "; " ?

'Disadvantaged Communities; Health & Human Services'

In [8]:
# first attempt: error due to extra space in some entries --> strip whitespace
category = pd.read_csv('category_table.csv')
cat = {}
for i in list(category.category):
    cat[i] = []
for i in grants_ongoing.Categories:
    temp = []
    for j in i.split('; '):
        temp.append(j)
    for key in cat:
        if key not in temp:
             cat[key].append(0)
        else:
             cat[key].append(1)
cat_df = pd.DataFrame(cat)
frames = [grants_ongoing, cat_df]
grants_ongoingBinary = pd.concat(frames, axis = 1)

In [9]:
# viewing index numbers for iloc
grants_ongoingBinary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 40 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   PortalID                                         125 non-null    int64  
 1   Status                                           125 non-null    object 
 2   LastUpdated                                      125 non-null    object 
 3   AgencyDept                                       125 non-null    object 
 4   Title                                            125 non-null    object 
 5   Type                                             125 non-null    object 
 6   LOI                                              125 non-null    object 
 7   Categories                                       125 non-null    object 
 8   Purpose                                          125 non-null    object 
 9   Description                     

In [10]:
# confirming for correctness
index = [7] + list(range(24,39))
grants_ongoingBinary.iloc[:,index]

,Categories,Health & Human Services,Disadvantaged Communities,Libraries and Arts,Education,"Employment, Labor & Training",Agriculture,Environment & Water,Food & Nutrition,"Housing, Community and Economic Development",Parks & Recreation,"Science, Technology, and Research & Development","Law, Justice, and Legal Services",Energy,Consumer Protection,Disaster Prevention & Relief
0,Health & Human Services,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Disadvantaged Communities; Libraries and Arts,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
2,Disadvantaged Communities; Education; Employme...,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0
3,Agriculture; Disadvantaged Communities; Educat...,0,1,0,1,1,1,1,1,1,1,1,0,0,0,0
4,Agriculture; Disadvantaged Communities; Educat...,0,1,0,1,1,1,1,1,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,Education,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
121,Disadvantaged Communities; Disaster Prevention...,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1
122,Education,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
123,Education,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


**Exploration of Grants' Descriptions**

credit to Manmohan Singh via

https://towardsdatascience.com/keyword-extraction-process-in-python-with-natural-language-processing-nlp-d769a9069d5c

for the useful information regarding keyword analysis.

In [11]:
# Test Code: checking behavior of iloc
print(list(grants_ongoingBinary.iloc[3,27:29]))
print(list(grants_ongoingBinary.iloc[3,[27,28,29]]))
type(grants_ongoingBinary.iloc[3,9])

[1, 1]
[1, 1, 1]


str

In [13]:
import spacy
nlp = spacy.load('en_core_web_sm')
# Idea: function that takes in categories (through a list in indices) and a dataset -> takes the descriptions of the entries with those categories -> returns keywords
# can be polished for later project and used with the larger dataset

# 3 categories: Disadvantaged communities; Education; Housing, Community and Economic Development
CatIn = [25,27,32] # index numbers for columns

def keywords(categories, data):
    # setting up password (to select entries with all desired categories)
    text = ""
    password = []
    for i in categories:
        password.append(1) 
        
    # checking password and obtaining descriptions
    for j in range(len(data)):
        if list(data.iloc[j,categories]) == password:
            text = text + ' ' + data.iloc[j,9] 
    
    # finding keywords
    out = nlp(text)
    print(out.ents)
    
# test case 1
keywords(CatIn, grants_ongoingBinary)

(DPR, Alliance Grants Program, $1.5 million, Alliance, Integrated Pest Management, IPM, IPM, IPM, Alliance Grant, Alliance Grants Program, 2023, DPR, DPR, the Pest Management Advisory Committee, PMAC, the Proposal Application, PMAC, PMAC, DPR, DPR, $1.5 million, 50,000, $1.5 million, California, California, the PMAC Charter, DPR, Agronomy, Air Quality, Automation, Community Health, Cover Crops, Cropping System, Crops, Ecology, Ecosystem, Fauna, Flora, Fumigant, Fungi, Fungicide, Herbicide,, Horticulture, Integrated Pest Management, Irrigation, Lakes, Land Management, Mating Disruption, Miticide, Natural Enemies, Oceans, Pathogens, Personal Protective Equipment, Pest, Pest Management, Pesticide, Pollinator, Pollution, Reduced-Risk, Rivers, Rodenticide, Soil Health, Streams, Sustainable, Training, Urban Pest Management, Vegetables, Vertebrate Pests, Virus, Watershed, Worker Health and Safety DPR's, Research Grants Program, IPM, DPR, Research Grants Program, a Proposal Application, DPR, t